In [13]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
import base64
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "aac123"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename,'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Aaron Campbell\'s Dashboard'))),
    html.Center(html.Img(src='data:/home/aaroncampbell_snhu/Desktop/Python Files/GraziosoSalvareLogo.png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Aaron Campbell\'s Dashboard'))),
    html.Hr(),
    html.Div(
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label':'Water Rescue','value':'Water'},
                {'label':'Mountain or Wilderness Rescue','value':'Mountain'},
                {'label':'Disaster Rescue or Individual Tracking','value':'Disaster'},
                {'label':'Reset','value':'Reset'}
                ],
            value='Reset',
            labelStyle={'display':'inline-block'}
       )
   ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type','value')])

def update_dashboard(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
    
    #IF filter for Water Rescue Dogs
    if value == 'Water':
        df = pd.DataFrame(list(shelter.read_all({
            {'$or':[{'breed':'Labrador Retriever Mix'},
                    {'breed':'Chesapeake Bay Retriever'},
                    {'breed':'Newfoundland'}]},
            {'$and':[{'sex_upon_outcome':'Intact Female'},
            {'$and':[{'age_upon_outcome_in_weeks':{'$gte':26}},
                     {'age_upon_outcome_in_weeks':{'$lte':156}}]}]}})))
    
    #IF filter for Mountain and Wilderness Rescue Dogs
    elif value == 'Mountain':
        df = pd.DataFrame(list(shelter.read_all({
            {'$or':[{'breed':'German Shepherd'},
                    {'breed':'Alaskan Malamute'},
                    {'breed':'Old English Sheepdog'},
                    {'breed':'Siberian Husky'},
                    {'breed':'Rottweiler'}]},
            {'$and':[{'sex_upon_outcome':'Intact Male'},
            {'$and':[{'age_upon_outcome_in_weeks':{'$gte':26}},
                     {'age_upon_outcome_in_weeks':{'$lte':156}}]}]}})))
    
    #IF filter for Disaster Rescue and Individual Tracking Dogs
    elif value == 'Disaster':
        df = pd.DataFrame(list(shelter.read_all({
            {'$or':[{'breed':'German Shepherd'},
                    {'breed':'Doberman Pinscher'},
                    {'breed':'Golden Retriever'},
                    {'breed':'Bloodhound'},
                    {'breed':'Rottweiler'}]},
            {'$and':[{'sex_upon_outcome':'Intact Male'},
            {'$and':[{'age_upon_outcome_in_weeks':{'$gte':20}},
                     {'age_upon_outcome_in_weeks':{'$lte':300}}]}]}})))
    
    #IF filter to display all animals
    elif value == 'Reset':
        df = pd.DataFrame.from_records(shelter.read_all({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
    return (data,columns)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(derived_viewport_data)
    return [
        dcc.Graph(            
            figure = px.pie(dff,names='breed')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app